In [1]:
import os 

from xcube.core.store import new_data_store
import zarr

In [2]:
# greate s3 data store
s3_store = new_data_store(
    "s3",
    root=os.environ["S3_USER_STORAGE_BUCKET"],
    max_depth=10,
    storage_options=dict(
        anon=False,
        key=os.environ["S3_USER_STORAGE_KEY"],
        secret=os.environ["S3_USER_STORAGE_SECRET"],
    ),
)

In [3]:
ds = s3_store.open_data("cubes/science/0.0.2/000000.zarr")
ds

<xarray.Dataset> Size: 55GB
Dimensions:                   (angle: 2, angle_x: 3, angle_y: 3, band: 12,
                               time: 1097, y: 1000, x: 1000, time_era5: 2983,
                               time_esa_wc: 2, time_lccs: 7)
Coordinates:
  * angle                     (angle) <U7 56B 'Zenith' 'Azimuth'
  * angle_x                   (angle_x) float64 24B 5.967e+05 ... 6.067e+05
  * angle_y                   (angle_y) float64 24B 5.665e+06 5.66e+06 5.655e+06
  * band                      (band) <U5 240B 'B01' 'B02' 'B03' ... 'B11' 'B12'
  * time                      (time) datetime64[ns] 9kB 2016-11-01T10:32:02.0...
  * time_era5                 (time_era5) datetime64[ns] 24kB 2016-11-01 ... ...
  * time_esa_wc               (time_esa_wc) datetime64[ns] 16B 2020-01-01 202...
  * time_lccs                 (time_lccs) datetime64[ns] 56B 2016-01-01 ... 2...
  * x                         (x) float64 8kB 5.967e+05 5.967e+05 ... 6.067e+05
  * y                         (y) float64 8kB 5.665e+06 5.665e+06 ... 5.655e+06
Data variables: (12/113)
    cloud_mask                (time, y, x) uint8 1GB dask.array<chunksize=(20, 500, 500), meta=np.ndarray>
    dem                       (y, x) float32 4MB dask.array<chunksize=(500, 500), meta=np.ndarray>
    era5_d2m_max              (time_era5) float32 12kB dask.array<chunksize=(2983,), meta=np.ndarray>
    era5_d2m_mean             (time_era5) float32 12kB dask.array<chunksize=(2983,), meta=np.ndarray>
    era5_d2m_median           (time_era5) float32 12kB dask.array<chunksize=(2983,), meta=np.ndarray>
    era5_d2m_min              (time_era5) float32 12kB dask.array<chunksize=(2983,), meta=np.ndarray>
    ...                        ...
    lccs_processed_flag       (time_lccs, y, x) float32 28MB dask.array<chunksize=(7, 500, 500), meta=np.ndarray>
    s2l2a                     (band, time, y, x) float32 53GB dask.array<chunksize=(12, 20, 500, 500), meta=np.ndarray>
    scl                       (time, y, x) uint8 1GB dask.array<chunksize=(20, 500, 500), meta=np.ndarray>
    solar_angle               (time, angle, angle_y, angle_x) float32 79kB dask.array<chunksize=(1, 2, 3, 3), meta=np.ndarray>
    spatial_ref               int64 8B ...
    viewing_angle             (time, band, angle, angle_y, angle_x) float32 948kB dask.array<chunksize=(1, 12, 2, 3, 3), meta=np.ndarray>
Attributes: (12/37)
    acknowledgment:               DeepFeatures project
    affine_transform:             [10.0, 0.0, 596705.0, 0.0, -10.0, 5664655.0...
    bbox_utm:                     [596721, 5654663, 606721, 5664663]
    bbox_wgs84:                   [10.379465602333127, 51.035364882577994, 10...
    center_utm:                   [5659658.217015095, 601716.1995742661]
    center_wgs84:                 [51.079407, 10.452089]
    ...                           ...
    site_id:                      0
    time_range_end:               2024-12-31
    time_range_start:             2016-11-01
    utm_zone:                     32U
    version:                      0.0.2
    xcube_stac_attrs:             {'data_url': 'https://documentation.dataspa...

In [4]:
compressor = zarr.Blosc(cname="zstd", clevel=5, shuffle=1)
encoding = {"s2l2a": {"compressor": compressor}}
s3_store.write_data(
    ds, f"cubes/science/0.0.2/000000_shuffle1.zarr", replace=True, encoding=encoding
)

'cubes/science/0.0.2/000000_shuffle1.zarr'

In [5]:
compressor = zarr.Blosc(cname="zstd", clevel=5, shuffle=2)
encoding = {"s2l2a": {"compressor": compressor}}
s3_store.write_data(
    ds, f"cubes/science/0.0.2/000000_shuffle2.zarr", replace=True, encoding=encoding
)

'cubes/science/0.0.2/000000_shuffle2.zarr'